# Poc of user feedback influence on the original model
## Create new entity
## Improve existing entity

In [1]:
from __future__ import unicode_literals, print_function
import spacy
import en_core_web_sm
import en_core_web_md
# import en_core_web_lg
import random
from pathlib import Path
from spacy.util import minibatch, compounding
import tarfile
import xml.etree.ElementTree as ET
import os
from pandas import ExcelWriter
import pandas as pd
from sklearn.model_selection import train_test_split
import timeit
from spacy.gold import GoldParse
from spacy.scorer import Scorer


# Preprocess the data

In [2]:
# unzip files
# fname = './2014 i2c2/track1 deid/training-PHI-Gold-Set2.tar.gz'
# tar = tarfile.open(fname, "r:gz")
# tar.extractall(path='./2014 i2c2/track1 deid')
# tar.close()
# fname = './2014 i2c2/track1 deid/2014_training-PHI-Gold-Set1.tar.gz'
# tar = tarfile.open(fname, "r:gz")
# tar.extractall(path='./2014 i2c2/track1 deid')
# tar.close()
# fname = './2014 i2c2/track1 deid/Track1-de-indentification.tar.gz'
# tar = tarfile.open(fname, "r:gz")
# tar.extractall(path='./2014 i2c2/track1 deid')
# tar.close()


# function to combine all the available and save to a new excel file
def xml2Excel(inDirectory = None, outFileName = None, outDirectory = None):   
    path = inDirectory
    rawTexts = []
    tags = []
    for filename in os.listdir(path):
        if not filename.endswith('.xml'): continue
        fullname = os.path.join(path, filename)
        tree = ET.parse(fullname)
        root = tree.getroot()
        rawText = root[0].text
        tag = []
        for child in root[1]:
            tag.append(child.attrib)
        rawTexts.append(rawText)
        tags.append(tag)
    fullPath = outDirectory + outFileName
    writer = ExcelWriter(outFileName)
    df = pd.DataFrame({'text': rawTexts, 'tags' : tags})
    df.to_excel(writer,'Sheet1', index = False)
    writer.save()
xml2Excel(inDirectory = './2014 i2c2/track1 deid/combined records/',
         outFileName = 'PythonExport.xlsx',
         outDirectory = './') 


In [2]:
df = pd.read_excel('./PythonExport.xlsx')

In [3]:
df_2006Train = pd.read_excel('./i2b2-2006Train.xlsx')

In [4]:
df_2006Test = pd.read_excel('./i2b2-2006Test.xlsx')

# What the data looks like

In [7]:
df_2006Train.iloc[0, 1]

"[{'start': '2', 'end': '11', 'text': '123547445', 'TYPE': 'ID'}, {'start': '12', 'end': '15', 'text': 'FIH', 'TYPE': 'HOSPITAL'}, {'start': '16', 'end': '23', 'text': '7111426', 'TYPE': 'ID'}, {'start': '24', 'end': '34', 'text': '47933/f911', 'TYPE': 'ID'}, {'start': '35', 'end': '41', 'text': '557344', 'TYPE': 'ID'}, {'start': '42', 'end': '47', 'text': '11/19', 'TYPE': 'DATE'}, {'start': '42', 'end': '47', 'text': '11/19', 'TYPE': 'DATE'}, {'start': '164', 'end': '169', 'text': '11/28', 'TYPE': 'DATE'}, {'start': '275', 'end': '280', 'text': 'Blind', 'TYPE': 'PATIENT'}, {'start': '442', 'end': '455', 'text': 'November 13th', 'TYPE': 'DATE'}, {'start': '459', 'end': '477', 'text': 'Sephsandpot Center', 'TYPE': 'HOSPITAL'}, {'start': '514', 'end': '527', 'text': 'November 15th', 'TYPE': 'DATE'}, {'start': '599', 'end': '642', 'text': 'Valtawnprinceel Community Memorial Hospital', 'TYPE': 'HOSPITAL'}, {'start': '682', 'end': '698', 'text': '16th of November', 'TYPE': 'DATE'}, {'start'

In [20]:
int(eval(df_2006Test.iloc[0, 1])[0]['end'])

11

In [10]:
df.head()

,text,tags
0,\n\n\nRecord date: 2106-02-12\n\nCampbell Orth...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
1,\n\n\nRecord date: 2108-03-14\n\n ...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
2,\n\n\nRecord date: 2109-09-14\n\n \nSeptember ...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
3,\n\n\nRecord date: 2111-10-10\n\n \n\n\n\nC...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."
4,\n\n\nRecord date: 2111-12-14\n\n \n\n\n\nNEUR...,"[{'id': 'P0', 'start': '16', 'end': '26', 'tex..."


In [11]:
df = pd.concat([df, df_2006Test, df_2006Train], axis = 0)

[entities available in spacy](https://spacy.io/api/annotation)


# Find all entity inside the raw data
## There are some categories which doesn't exist in the original spacy model and shouldn't the combined to the existing category, like: FAX, DEVICE, MEDICALRECORD, EMAIL, PHONE, IDNUM, USERNAME, PROFESSION, BIOID, AGE
## There are some categories which represent the same thing, but have different names, like: 'CITY' -> 'GPE'; 'COUNTRY' -> 'GPE'; 'DOCTOR' -> 'PERSON'; 'HEALTHPLAN' -> 'PRODUCT'; 'HOSPITAL' -> 'ORG'; 'LOCATION-OTHER' -> 'LOC'; 'ORGANIZATION' -> 'ORG'; 'PATIENT' -> 'PERSON'; 'STATE' -> 'GPE'; 'STREET' -> 'GPE'; 'URL' -> 'PRODUCT'; 'ZIP' -> 'GPE'
## DATE is the only category that doesn't need to change

In [14]:
print(findAllEntity(df),)

{'LOCATION-OTHER', 'COUNTRY', 'USERNAME', 'LOCATION', 'STATE', 'BIOID', 'HEALTHPLAN', 'PROFESSION', 'ZIP', 'FAX', 'ID', 'EMAIL', 'PATIENT', 'IDNUM', 'DOCTOR', 'DEVICE', 'PHONE', 'URL', 'ORGANIZATION', 'AGE', 'STREET', 'MEDICALRECORD', 'DATE', 'HOSPITAL', 'CITY'}


# Define some functions to manipulate the data

In [23]:
def transformDate(orginalData = None):
    texts = [text for text in orginalData['text']]
    transformTags = []
    for tags in orginalData['tags']:
        entity = {'entities': []}
        for tag in eval(tags):
            if tag['TYPE'] == 'DATE':
                entity['entities'].append((int(tag['start']), int(tag['end']), 'DATE'))
        transformTags.append(entity)
    transformedTrain = list(zip(texts, transformTags))
    return transformedTrain
# transform the train data to the format which the model accepted
def transform(orginalData = None):
    texts = [text for text in orginalData['text']]
    transformTags = []
    for tags in orginalData['tags']:
        entity = {'entities': []}
        for tag in eval(tags):
            if tag['TYPE'] == 'CITY' or tag['TYPE'] == 'COUNTRY' or tag['TYPE'] == 'STATE' or tag['TYPE'] == 'STREET' or tag['TYPE'] == 'ZIP':
                entity['entities'].append((int(tag['start']), int(tag['end']), 'GPE'))
            elif tag['TYPE'] == 'DOCTOR' or tag['TYPE'] == 'PATIENT':
                entity['entities'].append((int(tag['start']), int(tag['end']), 'PERSON'))
            elif tag['TYPE'] == 'HEALTHPLAN' or tag['TYPE'] == 'URL':
                entity['entities'].append((int(tag['start']), int(tag['end']), 'PRODUCT'))
            elif tag['TYPE'] == 'HOSPITAL' or tag['TYPE'] == 'ORGANIZATION':
                entity['entities'].append((int(tag['start']), int(tag['end']), 'ORG'))
            elif tag['TYPE'] == 'LOCATION-OTHER':
                entity['entities'].append((int(tag['start']), int(tag['end']), 'LOC'))
            else:
                entity['entities'].append((int(tag['start']), int(tag['end']), tag['TYPE']))
        transformTags.append(entity)
    transformedTrain = list(zip(texts, transformTags))
    return transformedTrain
# find all entity in the transformed data
def findAllEntityTransform(data = None):
    uniqueTag = set()
    for (_, tags) in data:
        for tag in tags['entities']:
            uniqueTag.add(tag[2])
    return uniqueTag

# find all entity type in the original data
def findAllEntity(data = None):
    uniqueTag = set()
    [[uniqueTag.add(tag['TYPE']) for tag in eval(tags)] for tags in data['tags']]
    return uniqueTag
# find one entity in the original data
def findOneEntity(data = None, entType = None):
    for tags in data['tags']:
        for tag in eval(tags):
            if tag['TYPE'] == entType:
                print(tag)

# Train Test split the data

In [28]:
train, test = train_test_split(df, test_size=0.1, random_state = 42)

# We can see, after the transformation, some categories belong to the original model, some categories are new.

In [47]:
#transform the raw to the format the model can accept, also combine some categories
transformedTrain = transform(train)
print(findAllEntityTransform(transformedTrain), )

{'ORG', 'AGE', 'USERNAME', 'MEDICALRECORD', 'LOC', 'EMAIL', 'PROFESSION', 'GPE', 'FAX', 'PRODUCT', 'BIOID', 'PHONE', 'DEVICE', 'IDNUM', 'DATE', 'PERSON'}


In [48]:
# new tags needed to be added into the model
uniqueTag = findAllEntityTransform(transformedTrain)

# final training data
TRAIN_DATA = transformedTrain

# there are 2 text files has been polluted, we will skip them and record them
textWithProblem = []

# The final train data, if integrated with SHADOW, this is the data format.

# The main function which is responsible to train the model.

In [49]:

def main(model=None, n_iter=3, output_dir=None):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model) # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")
    for LABEL in uniqueTag:
        ner.add_label(LABEL)  # add new entity label to entity recognizer
    # Adding extraneous labels shouldn't mess anything up
#     ner.add_label("VEGETABLE")
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                try:
                    nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
                except:
                    textWithProblem.append(texts)
                    pass
            print("Losses", losses)

    # test the trained model
    test_text =  test.iloc[0, 0]
    
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print('The entities:', ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
#         nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

#         # test the saved model
#         print("Loading from", output_dir)
#         nlp2 = spacy.load(output_dir)
#         # Check the classes have loaded back consistently
#         assert nlp2.get_pipe("ner").move_names == move_names
#         doc2 = nlp2(test_text)
#         for ent in doc2.ents:
#             print(ent.label_, ent.text)




# One example of how we trained the model

In [51]:
uniqueTag

{'AGE',
 'BIOID',
 'DATE',
 'DEVICE',
 'EMAIL',
 'FAX',
 'GPE',
 'IDNUM',
 'LOC',
 'MEDICALRECORD',
 'ORG',
 'PERSON',
 'PHONE',
 'PRODUCT',
 'PROFESSION',
 'USERNAME'}

In [56]:
nlp = spacy.load('./clinicalModel50/')
doc = nlp(test.iloc[10, 0])
spacy.displacy.render(doc, style='ent')

In [57]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(test.iloc[10, 0])
spacy.displacy.render(doc, style='ent')

In [50]:
start = timeit.default_timer()
#Your statements here
main(n_iter=50, output_dir='./clinicalModel50', model='en_core_web_sm')
stop = timeit.default_timer()
print('Time: ', (stop - start)/3600)

Loaded model 'en_core_web_sm'
Losses {'ner': 423427.4593170043}
Losses {'ner': 363197.13703751564}
Losses {'ner': 347324.87572836876}
Losses {'ner': 347453.1168746948}
Losses {'ner': 347663.90417051315}
Losses {'ner': 346866.2787351608}
Losses {'ner': 348285.6950626373}
Losses {'ner': 348255.3970565796}
Losses {'ner': 348454.9196534157}
Losses {'ner': 352130.07054519653}
Losses {'ner': 348392.14848423004}
Losses {'ner': 347215.50279283524}
Losses {'ner': 351018.39546871185}
Losses {'ner': 353010.1942367554}
Losses {'ner': 349687.6489186287}
Losses {'ner': 351394.32259988785}
Losses {'ner': 348306.7652115822}
Losses {'ner': 346430.7308278084}
Losses {'ner': 348677.36999988556}
Losses {'ner': 351362.75901532173}
Losses {'ner': 351424.6071653366}
Losses {'ner': 350407.2163658142}
Losses {'ner': 347625.9195218086}
Losses {'ner': 356634.57122564316}
Losses {'ner': 350918.90012550354}
Losses {'ner': 352559.73244524}
Losses {'ner': 348430.0164065361}
Losses {'ner': 348139.92509937286}
Losses 

In [7]:
start = timeit.default_timer()
#Your statements here
main(n_iter=50, output_dir='./clinicalModel100', model='./clinicalModel50')
stop = timeit.default_timer()
print('Time: ', (stop - start)/3600)

Loaded model './clinicalModel50'
Losses {'ner': 8861.119128109509}
Losses {'ner': 7247.018137876955}
Losses {'ner': 6698.23574395864}
Losses {'ner': 6077.538574642193}
Losses {'ner': 5876.503647861915}
Losses {'ner': 5526.6102505445615}
Losses {'ner': 5389.690935520634}
Losses {'ner': 5322.98828311476}
Losses {'ner': 5031.868627947443}
Losses {'ner': 4694.403815780844}
Losses {'ner': 4638.784839544633}
Losses {'ner': 4614.4632901044215}
Losses {'ner': 4700.501797199783}
Losses {'ner': 4646.467815173068}
Losses {'ner': 4455.470641309933}
Losses {'ner': 4455.021736184305}
Losses {'ner': 4405.018686967978}
Losses {'ner': 4175.297955599923}
Losses {'ner': 4231.158696277984}
Losses {'ner': 4091.5805302651324}
Losses {'ner': 4212.474289383827}
Losses {'ner': 4125.777456656617}
Losses {'ner': 4002.309083940011}
Losses {'ner': 3982.7125367260214}
Losses {'ner': 3987.294436380852}
Losses {'ner': 3861.723685390058}
Losses {'ner': 3871.564883480321}
Losses {'ner': 3729.871814871785}
Losses {'ner'

# Evaluate the accuracy of the model
## precision, recall and f1 score should be used to evaluate the performance
## a high recall is paramount given the risk of accidentally disclosing sensitive patient information if not all PHI are detected and removed from the document or replaced by fake data.
## A high precision is also desired to preserve the integrity of the documents, as a large number of false positives might obscure the meaning of the text or even distort it. 
## As the combined mean of precision and recall, the F1 score gives an overall measure for model performance that is frequently employed in the NLP literature.

# Train Accuracy and Test Accuracy of the model

# Original Model

# We will get precision, recall and F1 score for all the categories and for each category. 

In [20]:
# # entities are corrected predicted/# entities have been predicted in Original model  precision
# # entities are corrected predicted/# entities inside the document  recall 
# F1 score = 2*((precision*recall)/(precision + recall))
def evaluate(ner_model, examples):
    stats = ['ents_p', 'ents_r', 'ents_f', 'ents_per_type']
    scorer = Scorer() 
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_) #the raw text
        gold = GoldParse(doc_gold_text, entities=annot['entities']) # combine the raw text with corrected id
        pred_value = ner_model(input_) # a doc object, which contains the predicted entity
        scorer.score(pred_value, gold) # calculate the accuracy
    return {k: scorer.scores[k] for k in stats} # accumulate all the documents

ner_model = spacy.load('en_core_web_sm') # for spaCy's pretrained use 'en_core_web_sm'
resOriTrain = evaluate(ner_model, TRAIN_DATA)
print("train accuracy:")
print(resOriTrain)
resOriTest = evaluate(ner_model, transform(test))
print("test accuracy:")
print(resOriTest)

train accuracy:
{'ents_p': 6.7534690935303665, 'ents_r': 29.455688740420516, 'ents_f': 10.987722191680412, 'ents_per_type': {'CARDINAL': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'QUANTITY': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'GPE': {'p': 7.237635705669482, 'r': 25.862068965517242, 'f': 11.310084825636194}, 'PERSON': {'p': 21.5177065767285, 'r': 54.763948497854074, 'f': 30.895883777239707}, 'ORG': {'p': 3.163790493490766, 'r': 43.18181818181818, 'f': 5.895627644569816}, 'PRODUCT': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'DATE': {'p': 25.908096280087527, 'r': 23.56687898089172, 'f': 24.6820929747759}, 'MEDICALRECORD': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'ORDINAL': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'PERCENT': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'TIME': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'PROFESSION': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'AGE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'WORK_OF_ART': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'NORP': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'LAW': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'FAC': {'p': 0.0, 'r': 0.0,

## We can see precision is low, recall is higher, it shows the model tends to classify more token as sensitive information to get higher accuracy. Also we can see for each category, only those shared category can have correct prediction, because the model hasn't been exposed to the training data yet.

# Iterate 30 times, small model

In [19]:
ner_model = spacy.load('./clinicalModel/') # for spaCy's pretrained use 'en_core_web_sm'
resOriTrain = evaluate(ner_model, TRAIN_DATA)
print("train accuracy:")
print(resOriTrain)
resOriTest = evaluate(ner_model, transform(test))
print("test accuracy:")
print(resOriTest)

train accuracy:
{'ents_p': 89.50617283950618, 'ents_r': 82.62920023580271, 'ents_f': 85.93031572494125, 'ents_per_type': {'PERSON': {'p': 87.54385964912281, 'r': 85.66523605150215, 'f': 86.59436008676789}, 'MEDICALRECORD': {'p': 54.437869822485204, 'r': 69.17293233082707, 'f': 60.9271523178808}, 'DATE': {'p': 93.9068100358423, 'r': 93.86942675159236, 'f': 93.88811467250648}, 'ORG': {'p': 79.9410029498525, 'r': 55.99173553719008, 'f': 65.85662211421628}, 'PROFESSION': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'AGE': {'p': 95.48192771084338, 'r': 83.64116094986808, 'f': 89.17018284106892}, 'GPE': {'p': 82.06896551724138, 'r': 51.293103448275865, 'f': 63.12997347480107}, 'PHONE': {'p': 89.28571428571429, 'r': 61.72839506172839, 'f': 72.99270072992702}, 'IDNUM': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'USERNAME': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'EMAIL': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'DEVICE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'FAX': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'BIOID': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'L

# Iterate 50 times, small model

In [52]:
ner_model = spacy.load('./clinicalModel50/') # for spaCy's pretrained use 'en_core_web_sm'
resOriTrain = evaluate(ner_model, TRAIN_DATA)
print(resOriTrain)
resOriTest = evaluate(ner_model, transform(test))
print(resOriTest)

{'ents_p': 91.69864673803272, 'ents_r': 89.2120259382983, 'ents_f': 90.43824701195219, 'ents_per_type': {'DATE': {'p': 96.67458432304038, 'r': 97.21337579617835, 'f': 96.94323144104804}, 'PERSON': {'p': 92.44135534317984, 'r': 91.3304721030043, 'f': 91.88255613126078}, 'MEDICALRECORD': {'p': 67.61363636363636, 'r': 89.47368421052632, 'f': 77.02265372168284}, 'PROFESSION': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'AGE': {'p': 92.77777777777779, 'r': 88.12664907651715, 'f': 90.39242219215156}, 'GPE': {'p': 76.16580310880829, 'r': 63.36206896551724, 'f': 69.17647058823529}, 'ORG': {'p': 79.3103448275862, 'r': 80.78512396694215, 'f': 80.04094165813717}, 'PHONE': {'p': 91.48936170212765, 'r': 53.086419753086425, 'f': 67.1875}, 'IDNUM': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'USERNAME': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'EMAIL': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'DEVICE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'FAX': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'BIOID': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'LOC': {'p': 0.0, 'r': 0.0, '

# Iterate 100 times, small model

In [53]:
ner_model = spacy.load('./clinicalModel100/') # for spaCy's pretrained use 'en_core_web_sm'
resOriTrain = evaluate(ner_model, TRAIN_DATA)
print(resOriTrain)
resOriTest = evaluate(ner_model, transform(test))
print(resOriTest)

{'ents_p': 98.92072213500785, 'ents_r': 99.05678915307526, 'ents_f': 98.98870888561612, 'ents_per_type': {'DATE': {'p': 99.96019108280255, 'r': 99.96019108280255, 'f': 99.96019108280255}, 'PERSON': {'p': 97.43150684931507, 'r': 97.6824034334764, 'f': 97.55679382768967}, 'MEDICALRECORD': {'p': 97.70992366412213, 'r': 96.2406015037594, 'f': 96.96969696969695}, 'PROFESSION': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'AGE': {'p': 99.73684210526315, 'r': 100.0, 'f': 99.86824769433464}, 'ORG': {'p': 98.96907216494846, 'r': 99.17355371900827, 'f': 99.07120743034056}, 'GPE': {'p': 94.87179487179486, 'r': 95.6896551724138, 'f': 95.27896995708154}, 'PHONE': {'p': 98.78048780487805, 'r': 100.0, 'f': 99.38650306748467}, 'IDNUM': {'p': 94.44444444444444, 'r': 94.44444444444444, 'f': 94.44444444444444}, 'USERNAME': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'EMAIL': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'DEVICE': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'FAX': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'BIOID': {'p': 100.

In [ ]:
ner_model = spacy.load('./clinicalModel/') # for spaCy's pretrained use 'en_core_web_sm'
resOriTrain = evaluate(ner_model, TRAIN_DATA)
print("train accuracy:")
print(resOriTrain)

# Some visualization

In [7]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(test.iloc[0, 0])
spacy.displacy.render(doc, style = 'ent')

In [30]:
nlp = spacy.load('./clinicalModel/')
doc = nlp(test.iloc[0, 0])
spacy.displacy.render(doc, style = 'ent')